# TASK 1

# Scraping of data


In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
base_url='https://www.airlinequality.com/airline-reviews/british-airways'
pages=10
page_size=100


In [4]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews
   ---> 1000 total reviews


In [ ]:
df=pd.DataFrame()
df['reviews']=reviews
df.head()

In [ ]:
import os
data_directory='data'
os.makedirs(data_directory,exist_ok=True)
df.to_csv(os.path.join(data_directory, "BA_reviews.csv"), index=False)

# DATA COLLECTION


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
reviews=[]
stars=[]
date=[]
country=[]


In [ ]:
!pip install lxml


In [ ]:
for i in range(1, 36):
    page = requests.get(f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100")
    
    soup = BeautifulSoup(page.content, "lxml")
    
    for item in soup.find_all("div", class_="text_content"):
        reviews.append(item.text)
    
    for item in soup.find_all("div", class_ = "rating-10"):
        try:
            stars.append(item.span.text)
        except:
            print(f"Error on page {i}")
            stars.append("None")
            
    #date
    for item in soup.find_all("time"):
        date.append(item.text)
        
    #country
    for item in soup.find_all("h3"):
        country.append(item.span.next_sibling.text.strip(" ()"))

In [ ]:
stars1=stars[0:3500]
print(len(reviews), len(stars1), len(date), len(country))

In [ ]:
reviews

In [ ]:
df=pd.DataFrame({"reviews":reviews,"stars":stars1,"date":date,"country":country})

In [ ]:
df

In [ ]:
import os

In [ ]:
cwd=os.getcwd()

In [ ]:
df.to_csv(cwd+ "/BA_reviews.csv")

# DATA CLEANING


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
cwd=os.getcwd()
df=pd.read_csv(cwd+"/BA_reviews.csv",index_col=0)


In [ ]:
df.head()

In [ ]:
df['verified']=df.reviews.str.contains('Trip Verified')

In [ ]:
df

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
from nltk.corpus import stopwords
lemma=WordNetLemmatizer()
reviews_data=df.reviews.str.strip("✅ Trip Verified |")
corpus=[]
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

In [ ]:
df['corpus']=corpus

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df['date'] = pd.to_datetime(df['date'], format="%dth %B %Y", errors='coerce')

In [ ]:
df=df.dropna()

In [ ]:
df.stars.unique()

In [ ]:
df.loc[:,'stars']=df['stars'].astype(int)

In [ ]:
df.stars.value_counts()

In [ ]:
df.isnull().value_counts()

In [ ]:
df.shape

In [ ]:
df.reset_index(drop=True)

In [ ]:
df.to_csv(cwd+'/cleaned-BA-reviews.csv')

In [ ]:
pip install wordcloud

In [ ]:
import datetime as dt
from wordcloud import WordCloud,STOPWORDS

In [ ]:
df=pd.read_csv(cwd+'/cleaned-BA-reviews.csv')

In [ ]:
df=df.reset_index(drop=True)

AVERAGE RATING for BA 

In [ ]:
df.stars.mean()

What is the total counts for each ratings?

In [ ]:
df.stars.value_counts().plot(kind='bar')
plt.xlabel("Ratings")
plt.ylabel("total no of reviews")
plt.suptitle("total count for each rating")

In [ ]:
df_ratings=pd.DataFrame(df.stars.value_counts())
df_ratings=df_ratings.reset_index()
pct_values = (df_ratings.stars.values/ df_ratings.stars.values.sum() *100).tolist()
pct_values = [round(x,2) for x in pct_values]
df_ratings['pct_values'] = pct_values

In [ ]:
df_ratings

In [ ]:
df_ratings=df_ratings.rename(columns={'count':'total_counts'})
df_ratings

In [ ]:
clrs = ['Red' if (x ==  max(df_ratings.total_counts)) else 'grey' for x in df_ratings.total_counts ]

In [ ]:
clrs

In [ ]:
ax=sns.barplot(x=df_ratings.stars,y=df_ratings.total_counts,data=df_ratings,errwidth=0,palette=clrs)
ax.bar_label(ax.containers[0])
    
ax.set_xlabel("Ratings")
ax.set_ylabel("Total Number of reviews with that rating")
ax.set_title("Counts for each ratings")

All the unique countries

In [ ]:
print(f"{len(df.country.unique())} unique countries")

In [ ]:
df_country_review = pd.DataFrame(df.country.value_counts().head()).reset_index()
df_country_review.rename(columns={'index':'country','country':'total_reviews'}, inplace=True)

In [ ]:
df_country_review.rename(columns={'total_reviews':'country'}, inplace=True)
df_country_review

In [ ]:
df_country_review.plot(kind='bar',x='country')

Which country provided on average highest ratings?

In [ ]:
df_country_rating = pd.DataFrame(df.groupby('country').stars.mean().sort_values(ascending=False)).reset_index()
df_country_rating.rename(columns={'stars':'avg_rating'},inplace=True)
df_country_rating

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
ax1 = sns.barplot(x='country', y='avg_rating', data=df_country_rating[:12])
ax.bar_label(ax.containers[0])
ax.set_title("Top 12 Countries with avg highest rating provided to British Airways")

Time Series Analysis

In [ ]:
df.date=pd.to_datetime(df.date)
df.date

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(df, x='date', y="stars")
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
reviews=" ".join(df.corpus)

In [ ]:
plt.figure(figsize=(20,10))
stopwords=set(stopwords.words('english'))
wordcloud=WordCloud(height=600,width=600,max_font_size=100,max_words=100,stopwords=stopwords).generate(reviews)
plt.imshow(wordcloud,interpolation='bilinear')
plt.show()

There are many words that does not set the idea of whether the review is positive or negative. For example words like "passenger", "flight", etc. does not add conlcusive value hence we can include them in stopwords list.

In [ ]:
import nltk
from nltk.corpus import stopwords
reviews = " ".join(df.corpus)
plt.figure(figsize=(20,10))

stopwords = set(stopwords.words('english'))
stopwords.update(["ba","flight", "british","airway", "airline","plane", "told","also","passenger" \
                 "london", "heathrow", "aircraft", "could","even", "would"])
# Create and generate a word cloud image:
wordcloud = WordCloud(height=500,width=500,max_font_size=100, max_words=300, stopwords=stopwords).generate(reviews)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# WORD FREQUENCY

In [ ]:
from nltk import ngrams
from nltk.probability import FreqDist

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
words=reviews.split(" ")
stopwords = text.ENGLISH_STOP_WORDS.union(['flight', 'ba', "passenger","u", "london","airway","british","airline",\
                                           "heathrow","plane","lhr","review"])
new_words = [word for word in words if word not in stopwords]


In [ ]:
nlp_words=FreqDist(new_words).most_common(20)
all_fdist=pd.Series(dict(nlp_words))
all_fdist

In [ ]:
## Setting figure, ax into variables
fig, ax = plt.subplots(figsize=(15,8))

## Seaborn plotting using Pandas attributes + xtick rotation for ease of viewing
all_plot = sns.barplot(x=all_fdist.index, y=all_fdist.values, ax=ax)
all_plot.bar_label(all_plot.containers[0])
plt.xticks(rotation=30)

This gives us a glimpse of what customers are really talking about here. We see that Seat is most talked about the airline followed by "Service" and "food" which are all very important to customers in terms of service. However, we still do not know is how they are expressing about each of this service. To bring some significane to these terms we will use ngram plots to see if they are bad or good in experience.

Word frequency with N-gram

In [ ]:
import nltk.collocations as collocations
from nltk import FreqDist,bigrams

In [ ]:
reviews=" ".join(df.corpus)

In [ ]:
words=reviews.split(" ")

In [ ]:
new_words=[word for word in words if word not in stopwords]

def get_freq_dist(new_words,number_of_ngrams ):
    from nltk import ngrams
    
    ## Generate bigrams
    ngrams = ngrams(new_words, number_of_ngrams)

    ## Creating FreqDist
    ngram_fd = FreqDist(ngrams).most_common(40)

    ## Sort values by highest frequency
    ngram_sorted = {k:v for k,v in sorted(ngram_fd, key=lambda item:item[1])}

    ## Join bigram tokens with '_' + maintain sorting
    ngram_joined = {'_'.join(k):v for k,v in sorted(ngram_fd, key=lambda item:item[1])}

    ## Convert to Pandas series for easy plotting
    ngram_freqdist = pd.Series(ngram_joined)
    plt.figure(figsize=(10,10))
    ax = ngram_freqdist.plot(kind="barh")
    
    return ax


get_freq_dist(new_words,4)

We can see that there are very common positive terms regarding cabin crew. For example, cabin_crew_friendly_helpful, cabin_crew_friendly_attentive, cabin_crew_friendly_efficient, etc. So certainly customers are providing good reviews about cabin crew staff of British Airways.

However, there is one another approach that we can try to find the word frequencies which will give us better idea. We will group the reviews based on ratings. Say, we assume ratigs 1-3 are bad reviews, 4-6 are average/good experience and 7-10 indicates a great experience.

In [ ]:
ratings_1_3=df[df.stars.isin([1,2,3])]
ratings_4_6=df[df.stars.isin([4,5,6])]
ratings_7_10=df[df.stars.isin([7,8,9,10])]

reviews_1_3 = " ".join(ratings_1_3.corpus)
reviews_4_6 = " ".join(ratings_4_6.corpus)
reviews_7_10 = " ".join(ratings_7_10.corpus)


In [ ]:
#split the text of all reviews into a list of words
words_1_3 = reviews_1_3.split(" ")
words_4_6 = reviews_4_6.split(" ")
words_7_10 = reviews_7_10.split(" ")


In [ ]:
new_words_7_10 = [word for word in words_7_10 if word not in stopwords]

get_freq_dist(new_words_7_10,4)

In [ ]:
new_words = [word for word in words_4_6 if word not in stopwords]

get_freq_dist(new_words,4)

In [ ]:
new_words = [word for word in words_1_3 if word not in stopwords]

get_freq_dist(new_words,4)

In [ ]:
pip install textblob

Now we will use textblob library to define if the text is negative or positive and to what extent

In [ ]:
%%capture
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

#set a column Polarity with all 0 values initially
df['polarity']=0
for i in range(len(df.corpus)):
    sent= TextBlob(df.corpus[i])
    polarity  = sent.sentiment.polarity
    subjectivity  = sent.sentiment.subjectivity
    df['polarity'][i] = polarity

In [ ]:
# let's see how many texts are with positive comments

print(f"{df[(df['polarity'] >-0.2) & (df['polarity'] <0.2)].shape[0]} number of reviews between -0.2 and 0.2 polarity score")

print(f"{df[(df['polarity'] >-0.1) & (df['polarity'] <0.1)].shape[0]} number of reviews between -0.1 and 0.1 polarity score")